# advent of code 2024 - [day 7](https://adventofcode.com/2024/day/7)

### Parsing

In [ ]:
def gen_equations(file='input.txt'):
    """Generates tuples of integers"""
    file = open(file, 'r')
    for _, line in enumerate(file):
        els = tuple(line.split(":"))
        yield tuple((int(els[0]), tuple(int(x) for x in els[1].strip().split(' '))))

In [ ]:
file = "input.txt"
#file = "test.txt"

## Python solution

In [ ]:
from functools import cache

operators = {
    '*':lambda test_value, numbers, ops: test_value % numbers[-1] == 0 and is_valid_equation(test_value//numbers[-1], numbers[:-1], ops),
    '+':lambda test_value, numbers, ops: test_value - numbers[-1] >= 0 and is_valid_equation(test_value-numbers[-1], numbers[:-1], ops),
    '|':lambda test_value, numbers, ops: str(test_value)[1:].endswith(str(numbers[-1])) and is_valid_equation(int(str(test_value)[:-len(str(numbers[-1]))]), numbers[:-1], ops)
}

@cache
def is_valid_equation(test_value, numbers, ops):
    if len (numbers) == 1:
        return test_value == numbers[-1]
    else:
        return any(operators[k](test_value, numbers, ops) for k in operators if k in ops)
    
part1, part2 = (sum([x[0] if is_valid_equation(x[0], x[1], ops) else 0 for x in gen_equations(file)]) for ops in [('*','+'),('*','+','|')]) 
part1, part2

## Graph Solution

In [ ]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
            any_update = any_update or result.consume().counters._contains_updates
        if not any_update:
            break

In [ ]:
# cleaning
def clean():
    queries = [
    "CALL apoc.schema.assert({},{});",
    """MATCH (n)
    CALL {WITH n DETACH DELETE n}
    IN TRANSACTIONS OF 1000 ROWS;""",
    """CALL gds.graph.list()
    YIELD graphName
    WITH graphName AS g
    CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
    WITH graphName RETURN graphName;"""]

    for q in queries:
        gds.run_cypher(q, {})

### Ingestion


In [ ]:
equations = [{'test_value': test_value, 'numbers':list(numbers)} for test_value, numbers in gen_equations(file)]

In [ ]:
def ingest(equations):


    gds.run_cypher('CREATE INDEX equation_test_value_numbers IF NOT EXISTS FOR (eq:Equation) ON (eq.test_value, eq.number)')
    
    query_ingest = """
    UNWIND $equations AS eq
    CREATE (eq_node:Equation)
    SET eq_node += eq, eq_node:Root
    """

    gds.run_cypher(query_ingest, {'equations':equations})

In [ ]:
clean()
ingest(equations)

### Part 1

In [ ]:
sat_query = """
MATCH (eq:Equation&!(Satisfiable|Unsatisfiable))
WHERE size(eq.numbers) = 1 AND eq.numbers[0] = eq.test_value
SET eq:Satisfiable
"""

unsat_query = """
MATCH (eq:Equation&!(Satisfiable|Unsatisfiable))
WHERE size(eq.numbers) = 1 AND eq.numbers[0] <> eq.test_value
SET eq:Unsatisfiable
"""

addition_query = """
MATCH (eq:Equation&!(Satisfiable|Unsatisfiable)&!AdditionUnpacked)
SET eq:AdditionUnpacked
WITH eq
WHERE size(eq.numbers) > 1 AND eq.test_value > eq.numbers[-1]
MERGE (eq_plus:Equation {test_value: eq.test_value-eq.numbers[-1], numbers: eq.numbers[..-1]})
MERGE (eq)-[:SATISFIABLE_IF {op: '+'}]->(eq_plus)
"""

multiplication_query = """
MATCH (eq:Equation&!(Satisfiable|Unsatisfiable)&!MultiplicationUnpacked)
SET eq:MultiplicationUnpacked
WITH eq
WHERE size(eq.numbers) > 1 AND eq.test_value % eq.numbers[-1] = 0
MERGE (eq_mult:Equation {test_value: eq.test_value/eq.numbers[-1], numbers: eq.numbers[..-1]})
MERGE (eq)-[:SATISFIABLE_IF {op: '*'}]->(eq_mult)
"""

sat_backpropagation_query = """
MATCH (eq:Equation&!(Satisfiable|Unsatisfiable))-[:SATISFIABLE_IF]->(:Satisfiable)
SET eq:Satisfiable
"""

unsat_backpropagation_query = """
MATCH (eq:Equation&!(Satisfiable|Unsatisfiable)&MultiplicationUnpacked&AdditionUnpacked)
WHERE NOT EXISTS {(eq)-[:SATISFIABLE_IF]->(x:!Unsatisfiable)}
SET eq:Unsatisfiable
"""

rules =[sat_query, unsat_query, addition_query, multiplication_query, sat_backpropagation_query, unsat_backpropagation_query]
infer(rules)

In [ ]:
part1_query = """
MATCH (n:Root&Satisfiable)
RETURN sum(n.test_value) AS part1
"""
gds.run_cypher(part1_query, {})

### Part 2

In [ ]:
clean()
ingest(equations)

In [ ]:
concatenation_query = """
MATCH (eq:Equation&!(Satisfiable|Unsatisfiable)&!ConcatenationUnpacked)
SET eq:ConcatenationUnpacked
WITH eq
WHERE size(eq.numbers) > 1 AND eq.test_value > eq.numbers[-1] AND toString(eq.test_value) ENDS WITH toString(eq.numbers[-1])
WITH eq, toString(eq.test_value) AS tv_string, toString(eq.numbers[-1]) AS last_string
WITH eq, left(tv_string, size(tv_string)-size(last_string)) AS left_string
MERGE (eq_concat:Equation {test_value: toInteger(left_string), numbers: eq.numbers[..-1]})
MERGE (eq)-[:SATISFIABLE_IF {op: '|'}]->(eq_concat)
"""

unsat_backpropagation_query = """
MATCH (eq:Equation&!(Satisfiable|Unsatisfiable)&MultiplicationUnpacked&AdditionUnpacked&ConcatenationUnpacked)
WHERE NOT EXISTS {(eq)-[:SATISFIABLE_IF]->(x:!Unsatisfiable)}
SET eq:Unsatisfiable
"""

rules = [sat_query, unsat_query, addition_query, multiplication_query, concatenation_query, sat_backpropagation_query, unsat_backpropagation_query]
infer(rules)

In [ ]:
part2_query = """
MATCH (n:Root&Satisfiable)
RETURN sum(n.test_value) AS part1
"""
gds.run_cypher(part2_query, {})